# Reporting delay, weekly to daily interpolation, upscaling and other nowcasting tools – A playground for experimentation

In [1]:
import glob
import importlib
import os
from pathlib import Path
import re
import sys 

import epiweeks
import numpy as np
import pandas as pd
import plotly 
import plotly.graph_objects as go
import plotly.express as px

sys.path.append("../../../Rtrend_project/")
if os.path.basename(os.getcwd()) != "programs":
    os.chdir("..")  # Move to project root
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/pventura/NonCloudStorage/Flu_forecast/programs


# Organizing some historical data into a structured shape

For this, I can inspire on `epinowcast` format (example: https://package.epinowcast.org/articles/epinowcast.html)

In [4]:
# --- Load a set of target data files (from Flu), weekly,  
# =============================================

locations_fpath = Path("aux_data/locations.csv")
# truth_file_fmt = "hosp_data/season_2023-2024/truth_weekly_latest_{date}.csv"
truth_file_fmt = "hosp_data/flusight_target_2023-2024/target-hospital-admissions_{date}.csv"

truth_file_glob = truth_file_fmt.format(date="*")
truth_file_re = truth_file_fmt.format(date="(\d{4}-\d{2}-\d{2})")

# ---=====
# Look up all files matching pattern, and the dates
date_pattern = r"truth_weekly_latest_(\d{4}-\d{2}-\d{2}).csv"

truth_fpath_dict = dict()
for file_str in glob.glob(truth_file_glob):
    match = re.search(truth_file_re, file_str)
    if match:
        date_str = match.group(1)
        truth_fpath_dict[date_str] = Path(file_str)
        print(f"{date_str}: {pd.Timestamp(date_str).day_name()}")

# Sort, since glob does not
truth_fpath_dict = dict(sorted(truth_fpath_dict.items()))

# ====
# Load all files
print("Loading all truth data:")
truth_big_df = pd.concat(
    [pd.read_csv(fpath, parse_dates=["date"]) for fpath in truth_fpath_dict.values()], 
    keys=[pd.Timestamp(date_str) for date_str in truth_fpath_dict.keys()],
    names=["report_date"],
    axis=0,
) 

truth_big_df = (
    truth_big_df
    .reset_index()
    .set_index(["location", "report_date", "date"], drop=True)
    # .drop(["Unnamed: 0", "level_1"], axis=1)
    .drop(["X", "level_1"], axis=1)
    # .rename({"value": "counts", "date": "reference_date"})  # Epinowcast columns
)
truth_big_df["lag"] = truth_big_df.index.get_level_values("report_date") - truth_big_df.index.get_level_values("date")


# Locations file
locations_df = pd.read_csv(locations_fpath)

truth_big_df

<>:9: SyntaxWarning:

invalid escape sequence '\d'

<>:9: SyntaxWarning:

invalid escape sequence '\d'

/var/folders/3r/q6xn2s9s54dg6_ljmsn66vyhg2wvpm/T/ipykernel_93658/2608700270.py:9: SyntaxWarning:

invalid escape sequence '\d'



2023-11-11: Saturday
2023-11-04: Saturday
2024-02-24: Saturday
2024-04-20: Saturday
2023-12-09: Saturday
2024-03-02: Saturday
2024-03-16: Saturday
2024-01-13: Saturday
2023-12-23: Saturday
2024-01-06: Saturday
2024-04-27: Saturday
2023-12-30: Saturday
2023-10-21: Saturday
2024-03-09: Saturday
2023-09-30: Saturday
2024-03-23: Saturday
2023-10-07: Saturday
2023-12-02: Saturday
2023-12-16: Saturday
2024-01-27: Saturday
2023-09-23: Saturday
2024-04-13: Saturday
2024-04-06: Saturday
2024-01-20: Saturday
2023-10-14: Saturday
2023-10-28: Saturday
2024-03-30: Saturday
2024-02-03: Saturday
2024-02-17: Saturday
2023-11-18: Saturday
2023-11-25: Saturday
2024-02-10: Saturday
Loading all truth data:


location_name  value  weekly_rate      lag
location report_date date                                                  
02       2023-09-23  2023-09-23         Alaska      5     0.702814   0 days
01       2023-09-23  2023-09-23        Alabama     11     0.217229   0 days
05       2023-09-23  2023-09-23       Arkansas      6     0.197247   0 days
04       2023-09-23  2023-09-23        Arizona      9     0.122599   0 days
06       2023-09-23  2023-09-23     California     51     0.131151   0 days
...                                        ...    ...          ...      ...
50       2024-04-27  2022-02-12        Vermont      0     0.000000 805 days
53       2024-04-27  2022-02-12     Washington      8     0.103415 805 days
55       2024-04-27  2022-02-12      Wisconsin     29     0.492275 805 days
54       2024-04-27  2022-02-12  West Virginia      5     0.281694 805 days
56       2024-04-27  2022-02-12        Wyoming      2     0.345672 805 days

[170448 rows x 4 columns]

In [6]:
# =============
# The dataframe above is in the epinowcast format (except for column names): total cases in "date" as of "report_date"
# We can work with the "changes" distribution, as a function of the delay in weeks.

max_delay_weeks = 8  # Delays longer than this nr of weeks are disregarded. Is to say that, after 8 weeks, data is "golden true".

# ===============

df = truth_big_df.copy()
df: pd.DataFrame = df.loc[df["lag"] <= pd.Timedelta(f"{max_delay_weeks}w")]

# Export dataframe to test epinowcast in R
if True:
    (df
     .reset_index()
     .rename({"date": "reference_date", "value": "confirm"}, axis=1)
     .to_csv("r_code/test_data_for_epinowcast.csv", index=False))

# --- Table of delays: for each location and case date, has the total counts as a function of the delay
delay_table_df = df.reset_index().pivot(index=["location_name", "date"], columns="lag", values="value")
# Signature  ^^ : df.loc[(location_name, date), lag = counts

# --- Table of changes: same as above, but containing the change (positive or negative) between consecutive weeks
changes_table_df = delay_table_df.diff(axis=1)

changes_table_df

lag                       0 days  7 days  14 days  21 days  28 days  35 days  \
location_name date                                                             
Alabama       2023-07-29     NaN     NaN      NaN      NaN      NaN      NaN   
              2023-08-05     NaN     NaN      NaN      NaN      NaN      NaN   
              2023-08-12     NaN     NaN      NaN      NaN      NaN      NaN   
              2023-08-19     NaN     NaN      NaN      NaN      NaN      NaN   
              2023-08-26     NaN     NaN      NaN      NaN      NaN      0.0   
...                          ...     ...      ...      ...      ...      ...   
Wyoming       2024-03-30     NaN     0.0      0.0      0.0      0.0      NaN   
              2024-04-06     NaN     0.0      0.0      0.0      NaN      NaN   
              2024-04-13     NaN     0.0      1.0      NaN      NaN      NaN   
              2024-04-20     NaN     0.0      NaN      NaN      NaN      NaN   
              2024-04-27     NaN     NaN      NaN      NaN      NaN      NaN   

lag                       42 days  49 days  56 days  
location_name date                                   
Alabama       2023-07-29      NaN      NaN      NaN  
              2023-08-05      NaN      NaN      0.0  
              2023-08-12      NaN      0.0      0.0  
              2023-08-19      0.0      0.0      0.0  
              2023-08-26      0.0      0.0      0.0  
...                           ...      ...      ...  
Wyoming       2024-03-30      NaN      NaN      NaN  
              2024-04-06      NaN      NaN      NaN  
              2024-04-13      NaN      NaN      NaN  
              2024-04-20      NaN      NaN      NaN  
              2024-04-27      NaN      NaN      NaN  

[2120 rows x 9 columns]

In [7]:
# Some statistics and visualizations with the changes and counts
# ===============================

# import plotly.express as px
# import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Histogram
# ===============
col_names = [f"{round(x.days / 7)} weeks" for x in changes_table_df.columns]
fig = make_subplots(rows=len(changes_table_df.columns), cols=1, subplot_titles=col_names)

for i, col in enumerate(changes_table_df.columns):
    fig.add_trace(
        go.Histogram(x=changes_table_df[col], name=col_names[i]), 
        row=i+1, col=1
    )

fig.update_layout(height=2000,  title_text="Histograms for each column")
fig.show()



In [18]:
# Scatterplot: changes as a function of counts
# ===============================

fig = go.Figure()

fig.add_scatter(x=delay_table_df["0 days"], y=changes_table_df["7 days"], mode="markers", name="7 days")

# fig.update_layout(
#     xaxis=dict(type="log"),
#     yaxis=dict(type="log")
# )

fig.update_layout(
    # xaxis_type="log",#=dict(type="log"),
    # yaxis_type="log",
)
fig.show()
# Conclusion: not very helpful

In [32]:
# RELATIVE BACKFILLS – Divide by the report numbers at the reference week
# ==================
fig = go.Figure()

for loc_name in delay_table_df.index.get_level_values("location_name").unique():
    plot_sr = (changes_table_df["7 days"] / delay_table_df["0 days"]).xs(loc_name, level="location_name") 
    fig.add_histogram(
        x=plot_sr,
        name=f"7 days – {loc_name}",
    )


fig.update_layout(
    xaxis_title="Relative change (1 week)",
    yaxis_title="Frequency",
    title="Relative changes in NHSN flu hospitalization data, for the 2023-2024 season, 1-week after first report.",
)

fig.write_html("tmp_figs/relative_backfill_by_state.html")
